# Unit 07

`````{tab-set}
````{tab-item} Objective

::::{important} Objective

The objective of this hands-on session is to provide you with practical experience in applying supervised machine learning methods for classification tasks. Specifically, we will focus on k-nearest neighbor, logistic regression, and random forest classifier techniques.

In this example, we will analyze a dataset containing class labels for 40 dyes, with one molecule missing a class label. Our goal is to train classification models that not only accurately describe the training and test data but also predict the class label for the one sample where it is missing.

The reference data for training is obtained from [PhotochemCAD](https://www.photochemcad.com/).

::::

````


````{tab-item} Further Information

:::{admonition}Further Information


**pandas** and **pandas.DataFrame**
- [`pandas.DataFrames` (1)](https://pandas.pydata.org/docs/user_guide/10min.html)
- [`pandas.DataFrames` (2)](https://www.w3schools.com/python/pandas/pandas_dataframes.asp)
- [`pandas.DataFrames` (3)](https://www.datacamp.com/tutorial/pandas-tutorial-dataframe-python)

**sklearn**
- [k-Nearest Neighbours](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
- [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
- [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
- [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

- [`Pipelines`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
- [Decision Boundaries](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.DecisionBoundaryDisplay.html)
- [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
- [Pipelining and grid search](https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html#pipelining)

:::

````
`````


In [1]:
import os, sys
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import digichem as dc
from digichem import FP, Spectrum

In [2]:
# load data
with open('db_dyes_07.pickle', 'rb') as handle:
    df = pickle.load(handle)

df.head()

,dye_category,CAS,name,smiles,abs_max,solvent,charge,counter_ion,RDmol,MorganFP_r3_s512,MorganFP_r3_s1024,MorganFP_r4_s512,MorganFP_r4_s1024,abs_spec
K01,cyanines,977-96-8,"1,1'-Diethyl-2,2'-cyanine iodide",CCN1/C(=C/c2ccc3ccccc3[n+]2CC)C=Cc2ccccc21,524.25,ethanol,1,[I-],<rdkit.Chem.rdchem.Mol object at 0x7f264f9cdb70>,512 bit FP,1024 bit FP,512 bit FP,1024 bit FP,"220.0 to 900.0 nm, steps: 1.0 nm"
K02,cyanines,605-91-4,"1,1'-Diethyl-2,2'-carbocyanine iodide",CCN1/C(=C/C=C/c2ccc3ccccc3[n+]2CC)C=Cc2ccccc21,603.50,methanol,1,[I-],<rdkit.Chem.rdchem.Mol object at 0x7f264f9cdc60>,512 bit FP,1024 bit FP,512 bit FP,1024 bit FP,"220.0 to 900.0 nm, steps: 1.0 nm"
K03,cyanines,14187-31-6,"1,1'-Diethyl-2,2'-dicarbocyanine iodide",CCN1/C(=C/C=C/C=C/c2ccc3ccccc3[n+]2CC)C=Cc2ccc...,711.00,ethanol,1,[I-],<rdkit.Chem.rdchem.Mol object at 0x7f264f9cdd00>,512 bit FP,1024 bit FP,512 bit FP,1024 bit FP,"220.0 to 900.0 nm, steps: 1.0 nm"
K04,cyanines,4727-49-5,"1,1'-Diethyl-4,4'-cyanine iodide",CCN1C=C/C(=C\c2cc[n+](CC)c3ccccc23)c2ccccc21,592.00,ethanol,1,[I-],<rdkit.Chem.rdchem.Mol object at 0x7f264f9cdda0>,512 bit FP,1024 bit FP,512 bit FP,1024 bit FP,"220.0 to 900.0 nm, steps: 1.0 nm"
K05,cyanines,4727-50-8,"1,1'-Diethyl-4,4'-carbocyanine iodide",CCN1C=C/C(=C\C=C\c2cc[n+](CC)c3ccccc23)c2ccccc21,709.50,ethanol,1,[I-],<rdkit.Chem.rdchem.Mol object at 0x7f264f9cde40>,512 bit FP,1024 bit FP,512 bit FP,1024 bit FP,"220.0 to 900.0 nm, steps: 1.0 nm"


## A) Inspection and pre-processing of data

::::{tip} Task A1

Although training classifiers on multidimensional features (vectors, arrays, tensors) and multiple features is not problematic, the goal of this hands-on session is to train classifiers using only two features. 
These two features will be scalar values for each molecule, allowing us to easily visualize the decision boundaries and visually inspect the model's quality.

To achieve this, we will derive two features from the absorption spectrum of each dye. 

STEP 1 - Plotting Absorption Spectra
: Plot the absorption spectra of all dyes labeled 'acridines' and those labeled 'cyanines'. 

STEP 2 - Feature Design
: Analyze these spectra to identify common and distinguishing features. Based on your observations, devise a strategy to extract two features from the absorption spectra and store them in the DataFrame.

STEP 3 - Visualize Features
: Plot the distribution of the data points based on the two selected features. Assess if the features are suitable for classification tasks. (Note: Be sure to color-highlight the data points according to their respective class labels.)


:::{admonition} Variables and Features in the Database
:class: dropdown
CLASS OF DYE ('dye_category')
:  classification of the objects into acridines or cyanines

CAS ('CAS')
: CAS identifier of the objects

NAME ('name')
:  IUPAC name of the objects

SMILES ('smiles')
: SMILES string of each object

VISIBLE ABSORPTION MAXIMUM ('abs_max')
: localization of the experimental absorption maximum of a dye in the visible region (360-800 nm) in the solvent indicated in column 'solvent'

CHARGE ('charge')
: total charge of the molecules, the respective counter-ions are given in column 'counter_ion' 

Morgan Fingerprints (e.g. 'MorganFP_r3_s512')
: Morgan fingerprints from unit 06 that were generated using a radius of 3 or 4 (indicated by the 'r') and a final fingerprint-size of either 512 or 1024 bits (indicated by 's')

Principal Components (e.g. 'PC1_r3_s512')
: Two principal components (PC1 and PC2) from PCA analysis of the Morgan fingerprints, which were generated using a radius of 3 or 4 (indicated by the 'r') and a final fingerprint-size of either 512 or 1024 bits (indicated by 's')

Absorption Spectrum ('abs_spec')
: Experimental UVvis absorption spectrum of the dyes. The array consisting of the absorption wavelengths in the first column and the mean molar extinction coefficients in the second column, can be accesed by e.g. ```[x.spec for x in df['abs_spec']]```

:::


:::{admonition} Data Overview from pd.DataFrame
:class: dropdown

An overview of the number of objects and variables is returned by


```{code-block} python
:lineno-start: 1
:emphasize-lines: 1

df_dyes.count()
```

When interested in the counts of a certain dye class, one can for instance count values of a certain column. 
The values of a column can be accessed via:


```{code-block} python
:lineno-start: 2
:emphasize-lines: 2

df_dyes['dye_category'].values
```

Generally, one can just call the function `hist()` to plot histogramms of all numeric columns from a dataframe. 


```{code-block} python
:lineno-start: 3
:emphasize-lines: 3

df_dyes.hist()
```
:::


:::{admonition} Access spectra based on respective labels
:class: dropdown

Access the spectra from the DataFrame based on the respective labels to facilitate your analysis.
You can use the following code snippet to get rid of the absorption spectrum of the unlabelled data point (molecule with the `unknown` class.) and access only the absorption data of the cyanines and acridines.

```{code-block} python
:lineno-start: 4
:emphasize-lines: 4

spec_acridines_cyanines = df[df['dye_category'] != 'unknown']['abs_spec']
```

To get the array of wavelengths and respective extinction coefficients from the objects stored in the dataframe, you can do the following.
Generally, the array of the spectrum is accessed by the `spec` function:

```{code-block} python
:lineno-start: 5
:emphasize-lines: 5

arr_acridines_cyanines = [(idx, spec_acridines_cyanines.loc[idx].spec) for idx in spec_acridines_cyanines.index]
```
:::

:::{admonition} Lineplots with Matplotlib
:class: dropdown

Documentation: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

```{code-block} python
:lineno-start: 6
:emphasize-lines: 9

import matplotlib.pyplot as plt

for spectrum in arr_acridines_cyanines:
    plt.plot(spectrum[:, 0], spectrum[:, 1])
```
:::

::::

### Step 1 - Plotting

In [3]:
# get spectra of cyanines and acridines


In [1]:
# Plot spectra


### Step 2 - Feature Extraction


In [2]:
# Feature extraction: calculate the two features and dump them to the dataframe

### Step 3 - Visualization of features

In [3]:
# scatterplot of features


::::{tip} Task A2 

Prior to the supervised machine learning for classification, we have to split our data into a set of training and test data and furthermore prepare the unlabelled data point:

Please create a ```pd.DataFrame``` of the labeled data, including two columns for the features derived from the absorption spectra of the dyes.
Additionally, create a ```pd.Series``` for the respective labels (`acridines` or `cyanines`).

Ensure that the unlabeled data point is separated from the original dataset and is not included in either the training or test sets. 
Prepare this unlabeled data point in a similar pd.DataFrame format, as it will be used to predict the label using our trained model.

:::{admonition} Data Splitting (sklearn)
:class: dropdown

Before performing supervised machine learning for classification, it is crucial to properly split your data into training and test sets to evaluate the model's performance accurately. This ensures that the model generalizes well to new, unseen data. Improper data splitting can lead to misleading performance metrics and overfitting.

Typically, 70-80% of the data is used for training, and the remaining 20-30% is used for testing. This balance allows the model to learn effectively while having enough data to validate its performance.

To split your data in Python using scikit-learn, you can use the train_test_split function. Here is an example:

```{code-block} python
:lineno-start: 1
:emphasize-lines: 4

from sklearn.model_selection import train_test_split

# Assuming 'X' is your DataFrame of features and 'y' is your Series of labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 'test_size=0.2' means 20% of the data will be used for testing
```

:::

::::

In [4]:
# data splitting
from sklearn.model_selection import train_test_split


## B) Supervised Classification

The objective in section B is to train different classifiers to distinguish acridines and cyanines based on the two features extracted from the absorption data of the respective dyes.

:::::{tip} Task B1 - kNN

In a first step, we want to perform k-nearest neighbor classification.

::::{admonition} Step 1: Initialize a kNN model
:class: dropdown

The standard workflow in working with sklearn models, can be described as follows using the example of kNN:

STEP 1 - Import Libraries 
: We start by importing necessary classes from scikit-learn. These include KNeighborsClassifier for the kNN algorithm, Pipeline to streamline the preprocessing and modeling steps, and StandardScaler to standardize the features.

```{code-block} python
:lineno-start: 1
:emphasize-lines: 2,3

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
```

STEP 2 - Define the Scaler
: We create an instance of StandardScaler which standardizes features by removing the mean and scaling to unit variance. This is an essential preprocessing step to ensure the kNN algorithm performs well, as it relies on distance calculations.

```{code-block} python
:lineno-start: 4
:emphasize-lines: 5

# define scaling (mean=0, variance=1)
scaler = StandardScaler()
```

STEP 3 - Define the kNN Model
: We instantiate the KNeighborsClassifier with n_neighbors=2, which means the algorithm will consider the 2 nearest neighbors to make a classification decision.

```{code-block} python
:lineno-start: 6
:emphasize-lines: 7

# kNN considering 2 nearest neighhbors for classification
knn = KNeighborsClassifier(n_neighbors=2)
```

STEP 4 - Create a Pipeline
: We combine the scaler and kNN model into a pipeline. This ensures that the data is first scaled before being passed to the kNN classifier. The pipeline helps in maintaining a clean and efficient workflow.

```{code-block} python
:lineno-start: 8
:emphasize-lines: 9

# pipeline combining scaling and kNN model 
knn_clf = Pipeline(steps=[("scaler", scaler), ("knn", knn)])
```

STEP 5 - Fit the Model
: We fit the pipeline to the training data (X_train and y_train). In this context, fitting means the scaler will compute the mean and variance from the training data, which will be used to transform the data. The kNN model will then use this transformed data for making predictions.

```{code-block} python
:lineno-start: 10
:emphasize-lines: 11

# train model (remember: kNN has no training but is instance-based/lazy learner)
knn_clf.fit(X_train, y_train)
```

:::{admonition} Complete Code
:class: dropdown

```{code-block} python
:lineno-start: 1
:emphasize-lines: 12

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# define scaling (mean=0, variance=1)
scaler = StandardScaler()

# kNN considering 2 nearest neighhbors for classification
knn = KNeighborsClassifier(n_neighbors=2)

# pipeline combining scaling and kNN model 
knn_clf = Pipeline(steps=[("scaler", scaler), ("knn", knn)])

# train model (remember: kNN has no training but is instance-based/lazy learner)
knn_clf.fit(X_train, y_train)
```
:::

::::

::::{admonition} Step 2: Finding the Optimal k-Value for kNN Classification
:class: dropdown

Upon fitting the kNN model with a specific k-value, you can access the score, which indicates how accurately the model classifies the labeled data. For example, a score of 1.0 on the test set means the model correctly classified every test data point.

In python you can access the scores and predictions with the score and predcit functions:

```{code-block} python
:lineno-start: 12

# score (function takes features and labels and returns score)
knn_clf.score(X_test, y_test)

# prediction (function takes only features and returns a label)
knn_clf.score(X_sample)
```

To find the optimal k-value, perform kNN classification for various k-values and evaluate their scores. By comparing these scores, you can determine the best k-value for your model.
Instructions:

1. Perform kNN classification for a range of k-values.
2. Calculate the scores for both the training and test data for each k-value.
3. Visualize how the scores on the test and training data depend on the k-value.
3. Based on the visualization, choose the optimal k-value.

::::

::::{admonition} Step 3: Assessing the Model's Quality
:class: dropdown

Now that we have chosen the optimal k-value, we will train a kNN model using this value and analyze its performance. To do this, we will examine two different measures: the confusion matrix and decision boundaries.

1. Train the kNN model using the optimal k-value.
2. Generate and analyze the confusion matrix for the test data.
3. Visualize the decision boundaries of the trained model. Visualize the data point with unknown label. Is the model able to describe the class of this dye correct?

:::{admonition} Confusion Matrix
:class: dropdown

The confusion matrix is a table that summarizes the performance of a classification algorithm. It shows the number of correct and incorrect predictions made by the model compared to the actual classifications. Each row of the matrix represents the instances in an actual class, while each column represents the instances in a predicted class. This matrix helps to assess the model's quality by providing insights into its accuracy, precision, recall, and ability to distinguish between different classes.

```{code-block} python
:lineno-start: 1
:emphasize-lines: 6

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# create confusion matrix
cm = confusion_matrix(y_test, knn_clf.predict(X_test))

# convert array into pd.DataFrame
cm_matrix = pd.DataFrame(
    data=cm, 
    columns=['Actual: Acridine', 'Actual: Cyanine'], 
    index=['Predict: Acridine', 'Predict: Cyanine']
)

# visualize the confusion matrix as heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='rocket_r')
```
:::

:::{admonition} Decision Boundaries
:class: dropdown

Decision boundaries are the surfaces that separate different classes predicted by the model. In the context of kNN, these boundaries illustrate how the algorithm divides the feature space based on the training data. Visualizing decision boundaries helps to assess the model's quality by showing how well the model generalizes to new data and how it handles different regions of the feature space. It can reveal whether the model is overfitting or underfitting the data.

```{code-block} python
:lineno-start: 1
:emphasize-lines: 4,5,6,7,8,9,10

from sklearn.inspection import DecisionBoundaryDisplay

# compute and visualize descion boundaries from the classifier
disp = DecisionBoundaryDisplay.from_estimator(
    knn_clf, 
    X_all.values, 
    response_method="predict",
    xlabel='feature 1', 
    ylabel='feature 2',
)

# indicate class by color in scatterplot
color_mapping = {'cyanines': 'blue', 'acridines': 'orange'}
colors = [color_mapping[label] for label in y_all]

disp.ax_.scatter(X_all['feature 1'].values, X_all['feature 2'].values, c=colors)
```
:::
::::

:::::

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [6]:
# initial kNN model


In [7]:
# tuning of k


In [8]:
# using optimal k-value


In [9]:
# confusion matrix plot


In [10]:
# decision boundaries


:::::{tip} Task B2 - Logistic Regression

Now, we want to perform Logistic Regression classification.

::::{admonition} Step 1: Logistic Regression Pipeline
:class: dropdown

Set up a pipeline for a logistic regression model and fit the model with an initial set of parameters (e.g., default parameters).

```{code-block} python
:lineno-start: 1
:emphasize-lines: 1

from sklearn.linear_model import LogisticRegression
```
::::

::::{admonition} Step 2: Hyperparameter Tuning
:class: dropdown

After fitting the logistic regression model, we can tune the parameters, a process known as hyperparameter tuning. 
To find the optimal set of parameters, perform a grid search, testing different combinations of parameters and choosing the combination that gives the best score.

In the following example, we assume we have initialized a scaling function and a logistic regression model called scaler and logistic.
Next, define the grid of parameters you want to investigate, specifically a set of values for `C` (Inverse of regularization strength; smaller values specify stronger regularization).

```{code-block} python
:lineno-start: 2
:emphasize-lines: 16

from sklearn.model_selection import GridSearchCV

scaler = StandardScaler()
logistic = LogisticRegression(max_iter=10000, tol=0.1)

logr_clf = Pipeline(
    steps=[("scaler", scaler), ("logr", logistic)]
)

param_grid = {
    'logr__C': np.logspace(-4, 4, 20)
}

search = GridSearchCV(logr_clf, param_grid, n_jobs=2)
search.fit(X_train,y_train)
```

1. Learn about the parameters of a logistic regression [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
2. Add an additional property to the `param_grid` dictionary and perform the grid search using this new grid. Which combination of parameters gives the best results?

::::

::::{admonition} Step 3: Assessing the Model's Quality
:class: dropdown

Now that we have performed the grid search to identify the best performing hyperparameters, train a logistic regression model using these parameters and assess the model quality based on the confusion matrix and decision boundaries.

1. Train a logistic regression classifier using the best set of parameters from Step 2.
2. Generate and analyze the confusion matrix for the test data.
3. Visualize the decision boundaries of the trained model. Include the data point with the unknown label. Does the model correctly classify this dye?

::::
:::::

In [11]:
# Step 1: Initial model
from sklearn.linear_model import LogisticRegression


In [12]:
# Step 2: Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV


In [13]:
# Step 3 - Train model with best parameters


In [14]:
#Confusion Matrix and Decision boundaries


:::::{tip} Task B3 - Random Forest Classification

Now, we want to perform a Random Forest classification.

1. Setup the Pipeline for a Random Forest classification
2. Tune the follwoing hyperparameters: max_depth, min_samples_split, criterion
3. Visualize the Confusion matrix and decision boundaries of the model with the best parameters.

::::{admonition} Random Forest in sklearn

```{code-block} python
:lineno-start: 1

from sklearn.ensemble import RandomForestClassifier
```
::::

:::::

In [15]:
# step 1
from sklearn.ensemble import RandomForestClassifier


In [16]:
# step 2 Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV


In [17]:
# step 3
# model with tuned hyperparameters


:::::{tip} Task B4 - Choose a model for the task/dataset 

Now that we have configured three classifiers with optimized hyperparameters—namely k-Nearest Neighbors, Logistic Regression, and Random Forest Classifier—let's use all three models to predict the class of the unlabelled data point. Examine the details of the unlabelled molecule and determine if the models accurately classify its class.

Evaluate which model is best suited for performing classification based on the chosen two features. Then, repeat the classification task using the entire absorption spectrum of each dye as features. Analyze if this approach improves the model's performance.

:::::

In [18]:
# model predictions for unlabelled data


In [19]:
# visualize unlabelled mol


In [21]:
# repeat classification of your choice with whole absorption spectrum as feature vector
